In [ ]:
import pandas as pd
import numpy as np
import math
import re

import folium
#import wget

from sklearn import metrics

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

import threading
threading.activeCount()


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# creating instance of labelencoder
labelencoder = LabelEncoder()
# creating instance of minmaxscaler
scaler = MinMaxScaler()

from matplotlib import pyplot as plt 
import seaborn as sns

import urllib
from datetime import datetime
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
scontext = ssl.SSLContext(ssl.PROTOCOL_TLS)
scontext.verify_mode = ssl.VerifyMode.CERT_NONE

In [ ]:
#df_vehicle=pd.read_parquet('') 
df_vehicle=pd.read_csv('cars_us_used_10k_sample.csv')
df_vehicle.sample(5)

In [ ]:
df_vehicle['year'] = df_vehicle.year.astype('int')
df_vehicle = df_vehicle.loc[df_vehicle.year>=2002]

In [ ]:
def getStr(inVal):
    if inVal is None:
        return ''
    else:
        return str(inVal)
    
def getInt(inVal):
    if inVal is None:
        return ''
    elif isinstance(inVal,str):
        return ''
    else:
        return int(inVal)

def getVehicleImgs(row, errorLogFile):
    #print(row)
    ImgLinks = row.photo_links
    id = row.id
    vin = row.vin
    model=row.model
    make=row.make
    year = row.year
    target_folder_path='all_images' #
    
  
    BaseFileName=target_folder_path+id+'_'+getStr(vin)+'_'+getStr(make)+'_'+getStr(model)+'_'+str(getInt(year))+'_'
    
    ImgLinkList=''
    
    try:
        ImgLinkList = ImgLinks.split('|')
    except:
        errorLogFile.write('\nBad or no photo links at row :'+ id)
        return
    
    for i in range(len(ImgLinkList)):
        targetFileName=BaseFileName+str(i)+'.jpg'
        try:
            with urllib.request.urlopen(url = ImgLinkList[i].strip(), context=scontext) as f:
                with open(targetFileName,'wb') as nf:
                    nf.write(f.read())
        except:
            errorLogFile.write("\nCounln't download :"+ImgLinkList[i])    
    


In [ ]:
def downloadPhotos(inDF):
    errorLogFile=open('ErrorLogfile.log','a') 
    errorLogFile.write('Processing started at :'+str(datetime.now()))
    
    for index, row in inDF.iterrows():
        if index%100==0:
            print('Prcessed:',int(index), ' rows ', datetime.now())
        try:
            getVehicleImgs(row, errorLogFile)
        except:
            print('Exception while processing row :', index)
    print('Finished Processing at :', datetime.now())
    errorLogFile.write('\nProcessing finished at :'+str(datetime.now()))
    errorLogFile.close()

In [ ]:
downloadPhotos(df_vehicle.iloc[:])